# 一. 分析区域简介

我在本次数据清洗项目中选择的区域是中国贵州省贵阳市，是中国西南地区新兴的国家级大数据试验区。之所以选择该地区，一方面是因为它是我的家乡，我比较熟悉；另一方面课程中的案例研究使用的都是外国的城市，所以我选择一个中国的城市进行数据清洗和导入工作，想看看在不同文化环境下有什么样的不同。

地图位置及数据导出：

* [地图位置](https://www.openstreetmap.org/relation/2782246#map=9/26.7750/106.6965)
* [数据导出](http://overpass-api.de/api/map?bbox=105.1996,25.8469,108.1934,27.6981)

# 二. 数据整理

In [7]:
import xml.etree.cElementTree as ET
import pprint
import math
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# count all kinds of tags there
def count_tags(filename):
    tags = {}
    for _, elem in ET.iterparse(filename):
        if elem.tag in tags:
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
    return tags

def key_type(element, keys):
    if element.tag == "tag":
        k = element.attrib['k']
        if lower.search(k) is not None:
            keys['lower'] += 1
        elif lower_colon.search(k) is not None:
            keys['lower_colon'] += 1
        elif problemchars.search(k) is not None:
            keys['problemchars'] += 1
        else:
            keys['other'] += 1
        
    return keys

def count_keys(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

tags = count_tags('guiyang_china.osm')
pprint.pprint(tags)
print
keys = count_keys('guiyang_china.osm')
pprint.pprint(keys)

{'bounds': 1,
 'member': 24512,
 'meta': 1,
 'nd': 361665,
 'node': 342055,
 'note': 1,
 'osm': 1,
 'relation': 183,
 'tag': 66859,
 'way': 16618}

{'lower': 61283, 'lower_colon': 5271, 'other': 305, 'problemchars': 0}


整个地图数据有66M，总共840383行。可以从count_tags函数的返回中看到各种不同标签的数量。其中的bounds元素只有一条，它给出了地图数据中所有node元素的经纬度边界：

```
<bounds minlat="25.8469000" minlon="105.1996000" maxlat="27.6981000" maxlon="108.1934000"/>
```

后面可以通过这条信息审核一下地图数据中的经纬度坐标是不是落在这个范围内。另外从count_keys函数返回的信息可以看到所有的二级tag元素的key种类都有哪些，其中problemchars类型个数为0，表示地图数据中不存在有问题的key值。使用Udacity项目提供的代码（详见samplek.py）从原始数据中抽取各种大小的元素样本，运行data.py将样本数据转为csv文件并对数据进行人工观察。发现地图数据中至少存在以下问题：

1. 一些节点的经纬度不在有效边界范围内（多个数据项之间不匹配，不满足数据质量评估中的“一致性”原则）；
2. 一部分道路的英文名称出现了多种英文缩写或拼写错误（数据模式不统一，不满足数据质量评估中的“有效性”原则）；
3. 一些节点的name属性有中文，英文和中英文混合三种数据模式（数据模式不统一，不满足数据质量评估中的“有效性”原则）；

## 1. 节点经纬度问题

In [8]:
osm_file = 'guiyang_china.osm'

# number of nodes whose latitude is out of bounds
count_of_error_lat = 0
# number of nodes whose longitude is out of bounds
count_of_error_lon = 0
# the maximum difference of latitude
max_abs_lat_diff = -1
# the maximum difference of longitude
max_abs_lon_diff = -1
# the minimum difference of latitude
min_abs_lat_diff = float('inf')
# the minimum difference of longitude
min_abs_lon_diff = float('inf')

for event, element in ET.iterparse(osm_file, events=("start",)):
    # retrieve the latitude and longitude range from <bounds .../>
    if element.tag == 'bounds':
        minlat = float(element.attrib['minlat'])
        minlon = float(element.attrib['minlon'])
        maxlat = float(element.attrib['maxlat'])
        maxlon = float(element.attrib['maxlon'])

    # only node elements have latitude and longitude information
    if element.tag == 'node':
        lat = float(element.attrib['lat'])
        lon = float(element.attrib['lon'])
        if not (minlat <= lat <= maxlat):
            diff = min(math.fabs(lat-minlat), math.fabs(lat-maxlat))
            # print('node {} latitude {} out of bounds [{}, {}], diff {}'.format(element.attrib['id'], lat, minlat, maxlat, diff))
            count_of_error_lat += 1
            if diff > max_abs_lat_diff:
                max_abs_lat_diff = diff
            if diff < min_abs_lat_diff:
                min_abs_lat_diff = diff
        if not (minlon <= lon <= maxlon):
            diff = min(math.fabs(lat - minlat), math.fabs(lat - maxlat))
            # print('node {} longitude {} out of bounds [{}, {}], diff {}'.format(element.attrib['id'], lon, minlon, maxlon, diff))
            count_of_error_lon += 1
            if diff > max_abs_lon_diff:
                max_abs_lon_diff = diff
            if diff < min_abs_lon_diff:
                min_abs_lon_diff = diff

print('total number of error lat {}, lon {}'.format(count_of_error_lat, count_of_error_lon))
print('latitude diff range [{}, {}], longitude diff range [{}, {}]'.format(min_abs_lat_diff, max_abs_lat_diff, min_abs_lon_diff, max_abs_lon_diff))

total number of error lat 8497, lon 2703
latitude diff range [2.00000002337e-07, 4.3298604], longitude diff range [1.20000000017e-05, 4.3298604]


先来看看经纬度问题，简单写了一个脚本（参见代码check_latlon.py）来对总体数据进行检查，与bounds元素提供的边界值做对比。可以看到总体数据中有8497条数据纬度超出边界范围，另有2703条数据经度超出边界范围，数量上看还是很可观的，看来这部分数据存在问题。但进一步分析发现无论是经度还是纬度其误差范围最大都不超过5度，考虑到GPS等仪器产生的定位误差，我认为这部分数据误差并不严重，对后续的分析工作不造成影响，可以不做处理。

## 2. 道路的英文名称问题

与案例研究中提供的例子有所不同，贵阳市的地图数据中街道的名字只有一小部分出现在addr:street属性中，绝大部分出现在name，name:en和name:zh中，所以对这三种属性也要进行审核。运行audit.py中的代码，观察发现同一种道路的英文名称出现了两种不同的缩写或者拼写错误：高速公路（Expressway）出现了Expwy和Expy两种缩写，另外还出现了Exprssway这样的拼写错误。另外，对于东南西北方位标识也存在全称和缩写混用的情况。我在data.py代码中做了检查，使用update_name函数（参见audit.py）将所有的道路和方位的命名进行统一规范。

In [1]:
# a mapping for regular expression
mapping = { r'\bSt\b': r'Street ',
            r'\bSt.\b': r'Street ',
            r'\bAve\b': r'Avenue',
            r'\bRd.\b': r'Road',
            r'\bRd\b': r'Road',
            r'\bBlvd\b': r'Boulevard',
            r'\bExpy\b': r'Expressway',
            r'\bExpwy\b': r'Expressway',
            r'\bExprssway\b': r'Expressway',
            r'\bN\b': r'North',
            r'\bS\b': r'South',
            r'\bW\b': r'West',
            r'\bE\b': r'East',
            }

def update_name(name, mapping):
    for k, v in mapping.items():
        name = re.sub(k, v, name)
    return name

## 3. name，name:en以及name:zh属性不一致

name，name:en和name:zh属性也存在一些问题。首先name属性格式不一致，有的是中文名称，有的是英文名称，有的则包含了中文加英文名称。但是如果将name属性简单清洗只保留中文或英文名称我觉得都不太好，这样会丢失一部分信息，所以应当审核三种名称属性之间是否一致，并在后续的分析中以name:en代表的英文名称或name:zh代表的中文名称为准。这里运行了代码check_name.py来审核name与name:en以及name:zh之间的一致性，具体的方法是通过正则表达式从name中提取中文和英文名称，然后分别检查name:zh和name:en，看数据是否一致，得到的结果如下所示。

```
id: 3374791336 name: KFC name_en: 肯德基星力城餐厅 not match
id: 4241699404 name: 城关镇 name_zh: 金沙 not match
id: 4403132789 name: CCB name_en: China Construction Bank not match
id: 4432242971 name: ICBC name_en: Industrial and Commercial Bank of China not match
id: 4432242972 name: PSBC name_en: Postal Saving Bank of China not match
id: 4432242974 name: RCC name_en: Rural Credit Cooperatives not match
id: 4432242975 name: ABC name_en: Agricultural Bank of China not match
id: 4432242986 name: CCB name_en: China Construction Bank not match
id: 4432243508 name: ABC name_en: Agricultural Bank of China not match
id: 4432243509 name: PSBC name_en: Postal Saving Bank of China not match
id: 4432243518 name: CCB name_en: China Construction Bank not match
id: 4432243527 name: PSBC name_en: Postal Saving Bank of China not match
id: 4432243596 name: ICBC name_en: Industrial and Commercial Bank of China not match
id: 4432293827 name: ICBC name_en: Industrial and Commercial Bank of China not match
id: 4536059675 name: ABC name_en: Agricultural Bank of China not match
id: 4536064454 name: CCB name_en: China Construction Bank not match
id: 4536064456 name: ICBC name_en: Industrial and Commercial Bank of China not match
id: 4537061498 name: CCB name_en: China Construction Bank not match
id: 135187642 name: 官井南隧道 name_zh: 官井隧道 not match
id: 140408475 name: 六冲河 name_zh: 三岔河 not match
id: 140408483 name: 六冲河 name_zh: 三岔河 not match
id: 442640875 name: Xihgua Rd name_en: Xihua Rd not match
id: 467965990 name: 官井南隧道 name_zh: 官井隧道 not match
id: 482578820 name: Daxing Lu name_en: Daxing Road not match
id: 482578821 name: Daxing Lu name_en: Daxing Road not match
id: 489457817 name: Daxing Lu name_en: Daxing Road not match
```

可以看到其中有一部分是银行信息，名称是缩写，比如中国工商银行缩写为ICBC，这些数据在逻辑上是一致的，可以不做处理。有一条关于“城关镇”的数据，name_zh标注的是金沙，说明这里是金沙县的城关镇，这个也无问题。但有些项就有明显的错误了，比如第一项的英文名称标记的是中文名“肯德基星力城餐厅”；“六冲河”和“三岔河”是乌江的两条不同支流，其name属性和name:zh属性不匹配；洗花路英文名称应该为Xihua Rd而不是Xihgua Rd；大兴路英文名应该为Daxing Road而不是Daxing Lu等等，这些都属于录入错误。由于量不是很大且没有明显的规律性，选择手工修正。

# 三. 数据概况和探索

使用了数据库查询来提供数据库的统计概述，如：

* 文件大小
* 唯一用户的数量
* 节点和途径的数量
* 所选节点类型（如：咖啡店、商店等）的数量

此外，还计算了不在上述列表中的统计数据。对于 SQL 提交，某些查询使用超过一个表。

# 四. 数据改进建议

# 五. 结论

# 代码功能性

## 代码是否发挥期望的功能？
最终项目代码的功能性与项目文件中描述的一致。

# 项目可读性

## 项目是否使用良好的编码实践？
最终项目代码遵从直观、易于跟进的逻辑结构。

## 代码的注释是否有用且无赘言？
最终项目代码直观可读，且标有清晰注释。

# 你的地图中遇到的问题

## 项目是否记录了数据整理中遇到的挑战？
学生的回复表现出对审计过程、纠正或规范化数据方式的理解，包括应对特定于位置的问题，例如与语言或传统格式化方式相关。

## 数据是否通过编程进行了清理？
数据审计中遇到的某些问题通过编程进行了清理。

# 数据概述

## OSM XML 是否足够大？
OSM XML 未压缩时至少有 50 MB。

## 是否计算了数据集的概述统计？
使用了数据库查询来提供数据库的统计概述，如：

* 文件大小
* 唯一用户的数量
* 节点和途径的数量
* 所选节点类型（如：咖啡店、商店等）的数量

此外，还计算了不在上述列表中的统计数据。对于 SQL 提交，某些查询使用超过一个表。

## 是否对数据库查询进行了书面记录？
提交文件包含上述数据库查询及统计数据。

# 关于数据集的其他想法

## 是否包含额外改进建议？
提交文件包含一个或多个用于改进或分析数据的额外建议。

## 是否讨论了额外改进的益处和问题？
提交文件中详尽地讨论了实施改进的益处及一些预期的问题。

# 提交的全面性和简洁性

## 提交是否足够长，能够全面回答提出的问题？
提交文件足够长，能够全面回答所提出的问题，同时未给予不必要的细节。一般的良好准则是问题回复的长度在 3 到 6 页之间。